In [1]:
import re
import glob
import json

In [72]:
path = "../../graph/src/*.rs"

In [73]:
setup = """
use super::*;
use arbitrary::Arbitrary;

#[derive(Arbitrary, Debug, Clone)]
pub struct FromVecHarnessParams {
    pub directed: bool,
    pub directed_edge_list: bool,
    pub ignore_duplicated_nodes: bool,
    pub ignore_duplicated_edges: bool,
    pub verbose: bool,
    pub numeric_edge_types_ids: bool,
    pub numeric_node_ids: bool,
    pub numeric_edge_node_ids: bool,
    pub numeric_node_types_ids: bool,
    pub has_node_types: bool,
    pub has_edge_types: bool,
    pub has_weights: bool,
    pub name: String,
    pub edges: Vec<Result<StringQuadruple, String>>,
    pub nodes: Option<Vec<Result<(String, Option<Vec<String>>), String>>>,
}

"""

In [74]:
def read_file(path):
    with open(path, "r") as f:
        return f.read()
    
def read_files(path):
    return [
        read_file(file)
        for file in glob.glob(path)
    ]

In [75]:
files = read_files(path)

In [76]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text  # or whatever

def parse_arguments(arguments):
    flat = [
        remove_prefix(x.strip(), "mut").strip()
        for comb in arguments.split(":")
        for x in comb.rsplit(",", 1)
    ]
    
    return [
            (arg_name, arg_type)
            for (arg_name, arg_type) in zip(flat[::2], flat[1::2])
    ]

In [77]:
#  r"pub\s+fn\s+([^\(\{]+?)\((self|&\s*self|&mut\s+self|&'.\s+self)?([^\)\{]+?)\)\s*(:?->\s*([^\{]+?))\{"

In [80]:
functions = [
    x
    for file in files
    for x in re.findall(
        r"fn\s+([^\{]+?)\{", 
        "\n".join(
            re.findall(
                "impl\s+Graph\s+\{.+\}", 
                file.replace("\n", "")
            )
        ) or "", 
        re.MULTILINE
    )
    if not any(
        e in x
        for e in [
            "impl", 
            "Fn", 
            "Iter", 
            "NodeFileReader", 
            "EdgeFileReader", 
            "Graph", 
            "Compute_hash_Params",
            "&str"
            "&String",
            "SingleWalkParameters",
            "WalksParameters",
            "&[u32]",
            "WalkWeights",
            "Self",
            "&"
        ]
    )
]
len(functions)

2

In [84]:
functions

['validate_weight(weight: WeightT) -> Result<WeightT, String> ',
 'parse_weight(weight: String) -> Result<WeightT, String> ']

In [82]:
re.findall(r"\w+(<.+?>)\(\s*(self|&self|&\s*mut\s*self),\s+(.+)", functions[0])

[]

In [83]:
functions_info = {
    function_name:{
        "arguments":parse_arguments(arguments),
        "return_type":return_type.strip(),
    }
    for function in functions
    fo
}

SyntaxError: invalid syntax (<ipython-input-83-0998927f304a>, line 7)

In [42]:
print(json.dumps(functions_info, indent=4))

{
    "random_spanning_arborescence_kruskal": {
        "arguments": [
            [
                "&self",
                "random_state"
            ],
            [
                "EdgeT",
                "unwanted_edge_types"
            ],
            [
                "&Option<HashSet<Option<EdgeTypeT>>>",
                "verbose"
            ],
            [
                "bool",
                ""
            ]
        ],
        "return_type": "(HashSet<(NodeT, NodeT)>, Vec<NodeT>, NodeT, NodeT, NodeT)"
    },
    "spanning_arborescence_kruskal": {
        "arguments": [
            [
                "&self",
                "verbose"
            ],
            [
                "bool",
                ""
            ]
        ],
        "return_type": "(HashSet<(NodeT, NodeT)>, Vec<NodeT>, NodeT, NodeT, NodeT)"
    },
    "spanning_arborescence": {
        "arguments": [
            [
                "&self",
                "verbose"
            ],
            [
      

In [17]:
def filter_function(function_name, infos):
    return function_name in [
        "validate_weight",
        "parse_weight"
        
    ] or any(
        e in arg[1]
        for arg in infos["arguments"]
        for e in [
        ]
    )
        

In [18]:
structs = []
fields = []

template = """
#[derive(Arbitrary, Debug, Clone)]
struct %s {
%s
}
"""

for function_name, info in functions_info.items():
    
    # Check if the function has parameters (other than self)
    if len(info["arguments"]) == 0:
        continue
        
    if filter_function(function_name, info):
        print("\n\nSkipping ", function_name)
        print(info["arguments"])
        continue
        
    
    struct_name = function_name.capitalize() + "_Params"
    
    fields.append("\t{function_name} : {struct_name},".format(
        function_name=function_name, 
        struct_name=struct_name,
    ))
        
          
    struct = template%(
        struct_name,
        "\n".join([
            "\t{field_name} : {field_type},".format(
                field_name=arg_name,
                field_type=arg_type,
            )
        for arg_name, arg_type in info["arguments"]
        ])
    )
    
    structs.append(struct)
    
    
fields.append( "\tfrom_vec: FromVecHarnessParams,")

harness = template%(
    "TheUltimateFuzzer",
    "\n".join(fields)
)
structs.append(harness)
    
result = "\n".join(structs)



Skipping  validate_weight
[('weight', 'WeightT')]


Skipping  parse_weight
[('weight', 'String')]


Skipping  from_sorted_csv<S: Into<String>>
[('edge_file_reader', 'EdgeFileReader'), ('node_file_reader', 'Option<NodeFileReader>'), ('directed', 'bool'), ('directed_edge_list', 'bool'), ('edges_number', 'EdgeT'), ('nodes_number', 'NodeT'), ('name', 'S')]


Skipping  from_unsorted_csv<S: Into<String>>
[('edge_file_reader', 'EdgeFileReader'), ('node_file_reader', 'Option<NodeFileReader>'), ('directed', 'bool'), ('directed_edge_list', 'bool'), ('name', 'S')]


In [19]:
len(structs)

127

In [12]:
def build_call(function_name, infos):
    if filter_function(function_name, infos):
        return ""
    
    args = ", ".join([
        f"data.{function_name}.{arg[0]}"
        for arg in infos["arguments"]
    ])
    
    result = f"\tgraph.{function_name}({args})"
    
    if infos["return_type"].startswith("Result"):
        result += "?"
    
    return result + ";"

In [13]:
harness = """
pub fn mega_test(data: TheUltimateFuzzer) -> Result<(), String> {
    let mut graph = graph::Graph::from_string_unsorted(
        data.from_vec.edges.into_iter(),
        data.from_vec.nodes.map(|ns| ns.into_iter()),
        data.from_vec.directed,
        data.from_vec.directed_edge_list,
        data.from_vec.name,
        data.from_vec.ignore_duplicated_nodes,
        data.from_vec.ignore_duplicated_edges,
        data.from_vec.verbose,
        data.from_vec.numeric_edge_types_ids,
        data.from_vec.numeric_node_ids,
        data.from_vec.numeric_edge_node_ids,
        data.from_vec.numeric_node_types_ids,
        data.from_vec.has_node_types,
        data.from_vec.has_edge_types,
        data.from_vec.has_weights,
    )?;
    
    
%s

    Ok(())
}
"""%"\n".join(
    build_call(function_name, infos)
    for function_name, infos in functions_info.items()
)

In [14]:
with open("result.rs", "w") as f:
    f.write(setup + result + harness)